# CUADERNO ENTRENAMIENTO MODELO

# Indice

1. [Importar Datos](#1.-Importar-Datos)
2. [Train Test](#2.-Train-Test)
3. [Scaler](#3.-Scaler)
4. [Pipe Line](#4.-Pipe-Line)
5. [GadientBoostingClassifier](#5.-GadientBoostingClassifier)
6. [Hiperparametrizacion GBC](#6.-Hiperparametrizacion-GBC)
7. [Parametrizacion GBC](#7.-Parametrizacion-GBC)
8. [PCA](#8.-PCA)


In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, recall_score, accuracy_score,multilabel_confusion_matrix
import pickle
import os

##  1. Importar Datos

[indice](#Indice)

In [3]:
Train = pd.read_csv(os.path.join('../data', 'Train.csv'))

In [4]:
Test = pd.read_csv(os.path.join('../data', 'Test.csv'))

## 2. Train Test

[indice](#Indice)

In [ ]:
# Eliminadas: 'gender', 'age', 'diastolic', 'systolic', 'height_cm',

In [5]:
X_test = Test[['weight_kg','gender', 'age', 'diastolic', 'systolic', 'height_cm', 'body fat_%','gripForce', 'sit and bend forward_cm', 'sit-ups counts',
       'broad jump_cm']]
y_test = Test['class']

In [6]:
X = Train[['weight_kg','gender', 'age', 'diastolic', 'systolic', 'height_cm', 'body fat_%','gripForce', 'sit and bend forward_cm', 'sit-ups counts',
       'broad jump_cm']]

y = Train['class']

## 3. Scaler

[indice](#Indice)

from sklearn.metrics import make_scorer, recall_score

In [6]:
url= 'https://file+.vscode-resource.vscode-cdn.net/c%3A/Users/jamen/Documents/TheBridge/Alumno/JUAN%20ML/notebooks/#indice'

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.fit_transform(X_test)

In [10]:
print(X_scaled.shape)
print(X.shape)
print(X_test_scaled.shape)

(10714, 11)
(10714, 11)
(2679, 11)


## 4. Pipe Line
de ejercicio de Beast Cancer de 3.ML-1.supervisado -4.Pipeles-practica

[indice](#Indice)

In [17]:
pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ('classifier', RandomForestClassifier())
])

logistic_params = {
'selectkbest__k' : [1,2,4,6,7,8,9,10,11],
'classifier': [LogisticRegression(solver='liblinear')],
'classifier__penalty': ['l1','l2']
}

rf_params = {
# 'scaler' : [StandardScaler(), None],
'selectkbest__k' :[8,9,10],
'classifier': [RandomForestClassifier()],
'classifier__max_features': [2,3,4],
'classifier__max_depth': [9,10,11]
}

gb_params = {
'scaler' : [StandardScaler(), None],
'selectkbest__k' : [10,11],
'classifier': [GradientBoostingClassifier()],
'classifier__max_features': [4,5],
'classifier__max_depth': [7,8]
}

knn_params = {
'selectkbest__k' :[7,9],
'classifier': [KNeighborsClassifier()],
'classifier__n_neighbors': [2,3,5]
}

svm_params = {
'selectkbest__k' : [3,4],
'classifier': [SVC()],
'classifier__C': [50,60,70]
}

In [18]:
scorer = make_scorer(recall_score, average='macro')
search_space = [logistic_params, svm_params ,rf_params, gb_params ,knn_params,]

clf = GridSearchCV(estimator = pipe, param_grid = search_space, cv=3, scoring=scorer , n_jobs=-1)

In [19]:
clf.fit(X, y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('classifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid=[{'classifier': [LogisticRegression(solver='liblinear')],
                          'classifier__penalty': ['l1', 'l2'],
                          'selectkbest__k': [1, 2, 4, 6, 7, 8, 9, 10, 11]},
                         {'classifier': [SVC()], 'classifier__C': [50, 60, 70],
                          'selec...
                          'selectkbest__k': [8, 9, 10]},
                         {'classifier': [GradientBoostingClassifier(max_depth=7,
                                                                    max_features=4)],
                          'classifier__max_depth': [7, 8],
                          'classifier__max_features': [4, 5],
            

In [31]:
ruta_model = os.path.join("../models","trained_model_pip.pkl")

with open(ruta_model, 'wb') as archivo:
    pickle.dump(clf, archivo)

In [32]:
print("Best Score", clf.best_score_)
print("\t")
print("Best Params", clf.best_params_)
print("\t")
print("Best Estimators", clf.best_estimator_)

Best Score 0.7325548077617182
	
Best Params {'classifier': GradientBoostingClassifier(max_depth=7, max_features=4), 'classifier__max_depth': 7, 'classifier__max_features': 4, 'scaler': None, 'selectkbest__k': 10}
	
Best Estimators Pipeline(steps=[('scaler', None), ('selectkbest', SelectKBest()),
                ('classifier',
                 GradientBoostingClassifier(max_depth=7, max_features=4))])


In [35]:
y_pred_pip = clf.best_estimator_.predict(X_test)

In [36]:
print("Accuracy", accuracy_score(y_test,y_pred_pip))
print("Matriz Confusion ",multilabel_confusion_matrix(y_test,y_pred_pip))

Accuracy 0.7573721537887271
Matriz Confusion  [[[1810  184]
  [ 107  578]]

 [[1761  256]
  [ 233  429]]

 [[1870  159]
  [ 201  449]]

 [[1946   51]
  [ 109  573]]]



## 5. GadientBoostingClassifier


[indice](#Indice)

In [13]:
seed = 0
scorer = make_scorer(recall_score, average='macro')
                     
model_gbc = GradientBoostingClassifier(n_estimators=100, random_state=seed)

gbc_cv = cross_val_score(model_gbc, X, y, cv=10, scoring= scorer,error_score='raise')

print('CV',gbc_cv)
print('CV media',gbc_cv.mean())
print('CV desv',gbc_cv.std())

CV [0.71604992 0.68144605 0.70938325 0.69533893 0.72316257 0.72879883
 0.72510508 0.73558237 0.69368322 0.72524809]
CV media 0.7133798323541232
CV desv 0.016914598035810944


In [37]:
model_gbc.fit(X, y)

GradientBoostingClassifier(random_state=0)

In [40]:
ruta_model = os.path.join("../models","trained_model_gbc.pkl")

with open(ruta_model, 'wb') as archivo:
    pickle.dump(model_gbc, archivo)

In [41]:
y_pred_gbc = model_gbc.predict(X_test)

print("Accuracy", accuracy_score(y_test,y_pred_gbc))
print("Matriz Confusion ",multilabel_confusion_matrix(y_test,y_pred_gbc))

Accuracy 0.7282568122433744
Matriz Confusion  [[[1780  214]
  [  95  590]]

 [[1740  277]
  [ 256  406]]

 [[1855  174]
  [ 239  411]]

 [[1934   63]
  [ 138  544]]]


In [46]:
# para sacar la importancia de cada feature 

importancia = model_gbc.feature_importances_

for feature, importancia in zip(X.columns, importancia):
    print(f"{feature}: {importancia}")

weight_kg: 0.06720162827383427
gender: 0.020926607121788357
age: 0.06565674541992124
diastolic: 0.001850554873762779
systolic: 0.0014978676300325354
height_cm: 0.009604797503516073
body fat_%: 0.07895630932545923
gripForce: 0.03291280223404903
sit and bend forward_cm: 0.5143837581605198
sit-ups counts: 0.1915512543523271
broad jump_cm: 0.015457675104789426


Accuracy sin Scaler: 0.7286
Accuracy MinMaxScaler: 0.6162
Accuracy StandarScaler: 0.7248

## 6. Hiperparametrizacion GBC

[indice](#Indice)

In [7]:
model_gbcgs = GradientBoostingClassifier()
scorer = make_scorer(recall_score, average='macro')

parameters = {
    'scaler' : [StandardScaler(), None],
    'selectkbest__k' : [10,11],
    'classifier': [GradientBoostingClassifier()],
    'classifier__max_features': [4,5],
    'classifier__max_depth': [7,8]
    }

gb_gs = GridSearchCV(model_gbcgs, parameters, cv=5, scoring=scorer)

In [22]:
model_gbcgs.fit(X, y)

GradientBoostingClassifier()

In [23]:
ruta_model = os.path.join("../models","trained_model_gbcgs.pkl")

with open(ruta_model, 'wb') as archivo:
    pickle.dump(model_gbcgs, archivo)

In [52]:
y_pred = model_gbcgs.predict(X_test)

print("Accuracy", accuracy_score(y_test,y_pred))

Accuracy 0.7282568122433744


## 7. Parametrizacion GBC

[indice](#Indice)

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(max_depth=8,
                                 n_estimators=100,
                                 learning_rate=1.0,
                                 random_state=42)

gbc.fit(X, y)


y_pred_gbc = gbc.predict(X_test)
accuracy_score(y_test, y_pred_gbc)

0.7357222844344905

In [24]:
ruta_model = os.path.join("../models","trained_model_pa.pkl")

with open(ruta_model, 'wb') as archivo:
    pickle.dump(model_gbcgs, archivo)


## 8. PCA

[indice](#Indice)

In [42]:
from sklearn.cluster import KMeans

k = 3

kmeans = KMeans(n_clusters=k, random_state=101)

y_pred_pca = kmeans.fit_predict(X)


In [44]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_pred_pca, y)

array([[   0, 1278, 1258, 1124,  748],
       [   0,  254,  439,  625,  918],
       [   0, 1131,  988,  950, 1001],
       [   0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0]], dtype=int64)